Let $\tau_i$ denote the membrane time constant of neuron $i$. Then, up to the approximations related to the assumption of being in an Asynchronous Irregular state, we have the following forumalas for the mean and SD (standard deviation) of the input to different neurons. 

$\mu_i = \tau_i \sum_j W_{ij} r_j + h_i$

$\sigma^2_i = \tau_i \sum_j W_{ij}^2 r_j + \xi^2_i$

where $h_i$ and $\xi_i$ are the mean (which is positive) and SD of the external input, respectively.

And then the output firing rate of each neuron is given by 

$r_i = \Phi(\mu_i, \sigma_i) \qquad\qquad\qquad$     (1)

If I plug in the first two equations (where each is really $N$ equations) in the bottom one (again, really $N$ equations), then I obtain a system of $N$ equations for the $N$ unknowns, namely the $r_i$.  (part of) Your tasks is to solve this ssytem of equations. To do that, we will first promote these to time-dependent ODE's. These are auxiliary tools for us to find the fixed point, and are not really physical:

$T_i\frac{dr_i}{dt} + r_i = \Phi(\mu_i, \sigma_i)$

We will then solve these numerically using the Euler method (see my `utils_for_max.py`), and hope that the ODE's will converge onto a fixed point, which satisfies (1). The Euler discretization leads to

$r_i(t+1) = (1 - \alpha_i) r_i(t) + \alpha_i \Phi(\mu_i, \sigma_i)$

where I defined $\alpha_i = \frac{\delta t}{T_i}$. But to use `utils_for_max.Euler2fixedpt` you don't really need to do this, but you need to give it (as `dxdt`) the following:

$T_i^{-1} (-r_i + \Phi(\mu_i, \sigma_i))$ with the above expressions for $\mu$ and $\sigma$ plugged in. 


In the rest, I assume that you have already substituted the expressions for $\mu_i$ and $\sigma_i$, but won't write them explicitly for brevity.

In the 



In [1]:
import jax.numpy as np
import jax.random as jrand
import pandas as pd

from model import Model, theta_diff, block_matrix, random_matrix
from utils_for_max import Phi, Euler2fixedpt
import utils_for_max
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Define model with N_E and N_I
model = Model(8000, 2000)

# Assigning parameters J, P, w^2
J = np.log(np.array([[0.63, 0.6],
                    [0.32, 0.25]]))

P = np.log(np.array([[0.11, 0.11], 
                    [0.45, 0.45]]))

w = np.log(32) * np.ones([2,2])
            
model.set_parameters(J, P, w)

model.set_inputs(1, 0)

plt.scatter(np.array(range(model.N)) - model.N_E, model.h)

In [ ]:
model.generate_network()
model.solve_fixed_point()  # Is producing overflow error!
plt.scatter(np.array(range(model.N)) - model.N_E, model.r)

In [ ]:
plt.imshow(model.W, cmap='hot', interpolation='nearest')
plt.show()

In [ ]:
k_data_path = os.path.join(os.pardir, 'Data', 'K-Data.csv')
k_data = pd.read_csv(k_data_path)

v1_data = k_data[k_data.region == 'V1']
    
print(v1_data.m.unique())
m = v1_data.m.unique()[2]
v1_data = v1_data[v1_data.m == m]
print(len(v1_data))

v1_data.head()

In [ ]:
orientations = v1_data.grat_orientation.unique()
contrasts = v1_data.grat_contrast.unique()
spatial_frequencies = v1_data.grat_spat_freq.unique()
phases = v1_data.grat_phase.unique()
units = sorted(v1_data.u.unique())

In [ ]:
def oc_matrix(u, freq=None, phase=None):
    u_data = v1_data[v1_data.u == u]
    
    if freq is not None:
        u_data = u_data[u_data.grat_spat_freq == freq]
    if phase is not None:
        u_data = u_data[u_data.grat_phase == phase]
    
    M = np.zeros((len(orientations), len(contrasts)))
    Sd = np.zeros_like(M)
    
    for i, th in enumerate(orientations):
        ori_data = u_data[u_data.grat_orientation == th]
        for j, c in enumerate(contrasts):
            con_data = ori_data[ori_data.grat_contrast == c]
            
            M[i,j] = np.mean(con_data.response)
            Sd[i,j] = np.std(con_data.response)
            
    return M, Sd

In [ ]:
tuning_curves = np.array([])

for u in units:
    for i, spf in enumerate(spatial_frequencies):
        M, Sd = oc_matrix(u, spf)
        tuning_curves.append(M)

model.calculate_loss(tuning_curves)

###### 